## Protocol Analysis

It would probably be good to make these actual python libraries. Oh well.

In [ ]:
# First extract logs with "PERF" from the client/receiver logs

from pathlib import Path

filename="test.log"

if not Path(filename).exists():
    !cat ../logs/replica*.log ../logs/client*.log | grep PERF >{filename}

In [ ]:
# Basic log parsing

import re
import datetime 

def parse_time(line):
    match = re.search(f"([0-9]*:[0-9]*:[0-9]*.[0-9]*)", line)
    time_str = match.group(1);
    return datetime.datetime.strptime(time_str, "%H:%M:%S.%f")

def parse_tags(line):
    tags = {}
    line = line.split("PERF ")[1]
    for token in line.split():
        [tag, value] = token.split("=")
        tags[tag] = value
        try:
            tags[tag] = int(value)
        except ValueError as verr:
            pass
            
    return tags
        
def parse_line(line):
    time = parse_time(line)
    tags = parse_tags(line)

    tags["time"] = time
    return tags 

In [ ]:
events = []

with open(filename) as f:
    for line in f:
        events.append(parse_line(line))

In [ ]:
events = sorted(events, key=lambda x: x['time'])
commits = list(filter(lambda x: x["event"] == "commit", events))

In [ ]:
# Look at each client commit, make sure there are no two clients ops on the same seq
from collections import Counter

counts = Counter(c["seq"] for c in commits)
for seq in counts:
    if counts[seq] > 1:
        print(f"Sequence {seq} has different commits!")

In [ ]:
# Ensure all client operations are committed
n_clients = 2

for c_id in range(n_clients):
    c_commits = filter(lambda x: x["client_id"] == c_id, commits)
    seq = sorted(c["client_seq"] for c in c_commits)

    for x, y in zip(seq, seq[1:]):
        if (y-x) > 1:
            print(f"Client missed commits between {x}-{y}")


In [ ]:
# Get general stats
runtime = (commits[-1]["time"] - commits[0]["time"]).total_seconds()
print(f"Runtime: {runtime:.3f} s")
print(f"Total Throughput: {len(commits) / runtime:.2f} req/s")

print(f"Num commits: {len(commits)}")
print(f"Average latency: {sum(c['latency'] for c in commits) / len(commits):.0f} us")

fast = list(filter(lambda x: x["path"] == "fast", commits))
normal = list(filter(lambda x: x["path"] == "normal", commits))
slow = list(filter(lambda x: x["path"] == "slow", commits))

print("Fast path:")
print(f"\tNum commits: {len(fast)}")
print(f"\tAverage latency: {sum(c['latency'] for c in fast) / len(fast):.0f} us")


print("Normal path:")
print(f"\tNum commits: {len(normal)}")
print(f"\tAverage latency: {sum(c['latency'] for c in normal) / len(normal):.0f} us")

print("Slow path:")
print(f"\tNum commits: {len(slow)}")
print(f"\tAverage latency: {sum(c['latency'] for c in slow) / len(slow):.0f} us")



In [ ]:
# Get the throughput over periods of 100ms every 10ms  

import numpy as np

w_size = 1 #s
resolution = 0.05 #s

start_time = commits[0]["time"]
end = (commits[-1]["time"] - start_time).total_seconds()
events = [(c["time"] - start_time).total_seconds() for c in commits]

w_start = 0
i = 0
j = 0

event_counts = []

while w_start + w_size < end:
    while (events[i] < w_start):
        i += 1
    while (events[j] <= w_start + w_size):
        j += 1

    event_counts.append(j - i + 1)
    w_start += resolution


times = np.arange(0, end - w_size, resolution)
throughput = np.array(event_counts) / w_size


In [ ]:
# Plot throughput over time
import matplotlib.pyplot as plt

plt.plot(times, throughput, label='Throughput', alpha=0.5, linestyle='--')

plt.title('Throughput Over Time')
plt.xlabel('Time')
plt.ylabel('Throughput (Events per Window)')
plt.legend()
plt.show()